In [1]:
### Imports for the Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib as plt
import os

In [2]:
# get working directory and remove last folder
# TODO: make this more robust
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print('Working Directory: ', os.getcwd())

Working Directory:  n:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance


In [3]:
dtypes={
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'index':np.int32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'}


# os.chdir('N:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance')
dataset_df = pd.read_csv('data/raw/train.csv', dtype=dtypes, nrows= 20000)

In [4]:
#preprocessing function to add variables and rescale the time 
def adding_new_variables_rescaling(dataset_df):
    dataset_df = dataset_df.sort_values(['session_id','elapsed_time'])
    dataset_df['elapsed_time'] = dataset_df['elapsed_time']/1000
    group = dataset_df.groupby(['session_id','level'])['elapsed_time'].diff()
    group = group.fillna(value= 0)
    dataset_df= dataset_df.assign(difference_clicks = group)

    return dataset_df

In [5]:
#preprocessing to combine variables on level stage
#Function to clean the sequential data for the training of the model

#For that Function to work we need to specify the variables in Categorical and Numerical & Counting

CATEGORICAL = ['event_name', 'name', 'fqid', 'room_fqid', 'text_fqid', 'fullscreen', 'hq', 'music']
NUMERICALmean = ['hover_duration','difference_clicks']
NUMERICALstd = ['elapsed_time','page','room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration', 'difference_clicks']
COUNTING = ['index']
MAXIMUM = ['difference_clicks', 'elapsed_time']



def feature_engineer_steve(dataset_df):
    dfs = []
    tmp = dataset_df.groupby(['session_id','level'])["level_group"].first()
    tmp.name = tmp.name 
    dfs.append(tmp)
    for c in CATEGORICAL:
        if c not in ['fullscreen', 'hq', 'music']:
            tmp = dataset_df.groupby(['session_id','level'])[c].agg('nunique')
        else:
            tmp = dataset_df.groupby(['session_id','level'])[c].first().astype(int).fillna(0)
        dfs.append(tmp)
    for c in NUMERICALmean:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('mean')
        tmp.name = tmp.name + '_mean'
        dfs.append(tmp)
    for c in NUMERICALstd:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)    
    for c in COUNTING:
        tmp = 1+ dataset_df.groupby(['session_id','level'])[c].agg('max')- dataset_df.groupby(['session_id','level'])[c].agg('min') 
        tmp.name = tmp.name + '_sum_of_actions'
        dfs.append(tmp)
    for c in MAXIMUM:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('max')- dataset_df.groupby(['session_id','level'])[c].agg('min') 
        tmp.name = tmp.name + '_max'
        dfs.append(tmp)
    
    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id') 
    
# add Clicks per second afterwards cause we need the time for each level first
    dataset_df['clicks_per_second'] = dataset_df['index_sum_of_actions']/ dataset_df['elapsed_time_max']
 
    return dataset_df

In [6]:
#return the preprocessed data
##test data preprocessing with Subset of 5 million rows

dataset_df_added = adding_new_variables_rescaling(dataset_df)
dataset_df_level = feature_engineer_steve(dataset_df_added)
print(dataset_df_added)
print(dataset_df_level)

              session_id  index  elapsed_time          event_name       name  \
0      20090312431273200      0         0.000      cutscene_click      basic   
2      20090312431273200      2         0.831        person_click      basic   
3      20090312431273200      3         1.147        person_click      basic   
1      20090312431273200      1         1.323        person_click      basic   
4      20090312431273200      4         1.863        person_click      basic   
...                  ...    ...           ...                 ...        ...   
19995  20090317080721164   1364      2454.048  notification_click      basic   
19996  20090317080721164   1365      2455.479        object_click      basic   
19997  20090317080721164   1366      2456.860        object_hover  undefined   
19998  20090317080721164   1367      2457.726        object_click      basic   
19999  20090317080721164   1368      2459.226  notification_click      basic   

       level  page  room_coor_x  room_c

In [ ]:
# Export results
#dataset_df_added.to_csv('data/processed/data_df_added.csv')
#dataset_df_level.to_csv('data/processed/data_df_level.csv')

In [7]:
print(dataset_df_level.dtypes)

level                       uint64
level_group               category
event_name                   int64
name                         int64
fqid                         int64
room_fqid                    int64
text_fqid                    int64
fullscreen                   int32
hq                           int32
music                        int32
hover_duration_mean        float32
difference_clicks_mean     float64
elapsed_time_std           float64
page_std                   float64
room_coor_x_std            float64
room_coor_y_std            float64
screen_coor_x_std          float64
screen_coor_y_std          float64
hover_duration_std         float64
difference_clicks_std      float64
index_sum_of_actions         int32
difference_clicks_max      float64
elapsed_time_max           float64
clicks_per_second          float64
dtype: object


In [8]:
dataset_df_level.columns

Index(['level', 'level_group', 'event_name', 'name', 'fqid', 'room_fqid',
       'text_fqid', 'fullscreen', 'hq', 'music', 'hover_duration_mean',
       'difference_clicks_mean', 'elapsed_time_std', 'page_std',
       'room_coor_x_std', 'room_coor_y_std', 'screen_coor_x_std',
       'screen_coor_y_std', 'hover_duration_std', 'difference_clicks_std',
       'index_sum_of_actions', 'difference_clicks_max', 'elapsed_time_max',
       'clicks_per_second'],
      dtype='object')

In [9]:
#split the dataframe into three different ones depending on the level group
groups = dataset_df_level.groupby('level_group')

# Create a dictionary to store the resulting dataframes
result = {}

# Loop over each group
for name, group in groups:
    # Add the group to the result dictionary
    result[name] = group

# Access the resulting dataframes using their keys
df_0_4 = result['0-4']
df_5_12 = result['5-12']
df_13_22 = result['13-22']


In [24]:
import pandas as pd
import numpy as np

# sample data with NaN values
df = pd.DataFrame({
    'session_id': [1, 1, 2, 2],
    'level_group': ['0-4', '0-4', '0-4', '0-4'],
    'col1': [1, 2, np.nan, 4],
    'col2': [5, 6, 7, 8],
    'col3': [9, 10, 11, np.nan]
})

# get the unique session_ids and columns
session_ids = df['session_id'].unique()
cols = ['session_id'] + [f'{col}_{i+1}' for i in range(len(session_ids)) for col in ['col1', 'col2', 'col3']]

# create a new dataframe to hold the flattened data
new_df = pd.DataFrame(columns=cols)

# loop through each session_id
for i, session_id in enumerate(session_ids):
    df_session = df[df['session_id'] == session_id].reset_index(drop=True)
    # combine the columns into a single row
    row_data = [df_session[col].iloc[i] for i in range(len(df_session)) for col in ['col1', 'col2', 'col3']] 
    # add None values to the row if necessary to make it the same length as the columns
    if len(row_data) < len(cols) - 1:
        row_data += [None] * (len(cols) - len(row_data) - 1)
    # add the session_id to the beginning of the row
    row_data = [session_id] + row_data
    # add the row to the new dataframe
    new_df.loc[len(new_df)] = row_data

# fill NaN values with np.nan
new_df = new_df.fillna(np.nan)

# convert the columns back to their original datatypes
for col in ['col1', 'col2', 'col3']:
    new_df[[f'{col}_{i+1}' for i in range(len(session_ids))]] = new_df[[f'{col}_{i+1}' for i in range(len(session_ids))]].astype(df[col].dtype, errors='ignore')

# sort the columns
new_df = new_df[cols]

print(new_df)


   session_id  col1_1  col2_1  col3_1  col1_2  col2_2  col3_2
0         1.0     1.0       5     9.0     2.0       6    10.0
1         2.0     NaN       7    11.0     4.0       8     NaN


In [37]:
import pandas as pd
import numpy as np

def flatten_df(df):
    # create a new index col
    df = df.reset_index()
    df['_index'] = df.index + 1
    df = df.set_index('_index')
    df = df.drop(["level_group"], axis= 1)

    # get the unique session_ids and columns
    session_ids = df['session_id'].unique()
    cols = ['session_id'] + [f'{col}_{i+1}' for i in range(len(session_ids)) for col in df.columns if col != 'session_id']

    # create a new dataframe to hold the flattened data
    new_df = pd.DataFrame(columns=cols)

    # loop through each session_id
    for i, session_id in enumerate(session_ids):
        df_session = df[df['session_id'] == session_id].reset_index(drop=True)
        # combine the columns into a single row
        row_data = [df_session[col].iloc[i] if pd.notnull(df_session[col].iloc[i]) else np.nan for i in range(len(df_session)) for col in df.columns if col != 'session_id'] 
        # add None values to the row if necessary to make it the same length as the columns
        if len(row_data) < len(cols) - 1:
            row_data += [np.nan] * (len(cols) - len(row_data) - 1)
        # add the session_id to the beginning of the row
        row_data = [session_id] + row_data
        # add the row to the new dataframe
        new_df.loc[len(new_df)] = row_data

    # fill NaN values with np.nan
    new_df = new_df.fillna(np.nan)

    # convert the columns back to their original datatypes
    for col in df.columns:
        if col != 'session_id':
            new_df[[f'{col}_{i+1}' for i in range(len(session_ids))]] = new_df[[f'{col}_{i+1}' for i in range(len(session_ids))]].astype(df[col].dtype, errors='ignore')

    # sort the columns
    new_df = new_df[cols]

    return new_df


In [47]:
new_df = flatten_df(df_5_12)

ValueError: Length of values (185) does not match length of index (415)

In [42]:
df = pd.DataFrame({
    'session_id': [1, 1, 2, 2],
    'level_group': ['0-4', '0-4', '0-4', '0-4'],
    'col1': [1, 2, 3, 4],
    'col2': [5, 6, 7, 8],
    'col3': [9, 10, 11, 12]
})

new_df_2 = flatten_df(df)

print(new_df_2)


   session_id  index_1  col1_1  col2_1  col3_1  index_2  col1_2  col2_2  \
0           1        0       1       5       9        1       2       6   
1           2        2       3       7      11        3       4       8   

   col3_2  
0      10  
1      12  


In [ ]:
df_0_4 = df_0_4.reset_index()
df_0_4 = df_0_4.set_index('index')
df_0_4 = df_0_4.drop(["level_group"], axis= 1)

In [43]:
import pandas as pd
import numpy as np

def flatten_df(df):
    # create a new index col
    df = df.reset_index()
    df['_index'] = df.index + 1
    df = df.set_index('_index')
    df = df.drop(["level_group"], axis= 1)

    # get the unique session_ids and columns
    session_ids = df['session_id'].unique()
    cols = ['session_id'] + [f'{col}_{i+1}' for i in range(len(session_ids)) for col in df.columns if col != 'session_id']

    # create a list to hold the flattened data
    rows = []

    # loop through each session_id
    for session_id in session_ids:
        df_session = df[df['session_id'] == session_id].reset_index(drop=True)
        # combine the columns into a single row
        row_data = [df_session[col].iloc[i] if pd.notnull(df_session[col].iloc[i]) else np.nan for i in range(len(df_session)) for col in df.columns if col != 'session_id'] 
        # add None values to the row if necessary to make it the same length as the columns
        if len(row_data) < len(cols) - 1:
            row_data += [np.nan] * (len(cols) - len(row_data) - 1)
        # add the session_id to the beginning of the row
        row_data = [session_id] + row_data
        # append the row to the list
        rows.append(row_data)

    # create the new dataframe from the list
    new_df = pd.DataFrame(rows, columns=cols)

    # fill NaN values with np.nan
    new_df = new_df.fillna(np.nan)

    # convert the columns back to their original datatypes
    for col in df.columns:
        if col != 'session_id':
            new_df[[f'{col}_{i+1}' for i in range(len(session_ids))]] = new_df[[f'{col}_{i+1}' for i in range(len(session_ids))]].astype(df[col].dtype, errors='ignore')

    # sort the columns
    new_df = new_df[cols]

    return new_df


In [46]:
import pandas as pd
import numpy as np

def flatten_df(df):
    # create a new index col
    df = df.reset_index()
    df['_index'] = df.index + 1
    df = df.set_index('_index')
    df = df.drop(["level_group"], axis= 1)

    # group by session_id
    grouped = df.groupby('session_id')

    # get the unique session_ids and columns
    session_ids = grouped.groups.keys()
    cols = ['session_id'] + [f'{col}_{i+1}' for i in range(len(session_ids)) for col in df.columns if col != 'session_id']

    # create a new dataframe to hold the flattened data
    new_df = pd.DataFrame(columns=cols)

    # define a function to combine columns into a single row
    def combine_cols(df_session):
        row_data = []
        for col in df.columns:
            if col != 'session_id':
                row_data.extend(df_session[col].tolist())
        row_data = [df_session['session_id'].iloc[0]] + row_data
        return pd.Series(row_data, index=cols)

    # apply the function to each session_id group and concatenate the results
    new_df = pd.concat(grouped.apply(combine_cols))

    # convert the columns back to their original datatypes
    dtypes = {col: df[col].dtype for col in df.columns if col != 'session_id'}
    new_df = new_df.astype(dtypes, errors='ignore')

    return new_df
